In [2]:
import cvxpy as cp
import numpy as np
import math
import control as ct


from IPython.display import display
from itertools import product
from itertools import permutations

In [3]:
def show_matrix(name, matrix, decimal_places=2):
    """
    Apresenta uma matriz com a quantidade de casas decimais desejadas.

    Parâmetros:
    ---
    - matrix: numpy.ndarray, a matriz a ser apresentada.
    - casas_decimais: int, o número de casas decimais desejadas (padrão é 2).
    """
    pattern = "{:." + str(decimal_places) + "e}"

    def format_elem(elem):
        return pattern.format(elem)
    
    width = [max(map(len, map(format_elem, coluna))) for coluna in matrix.T]

    print(name, "=")

    nspaces = sum(width) + 2 * matrix.shape[1]

    print("    ┌" + " " * nspaces + "┐")
    for line in matrix:
        formatted_line = "  ".join(format_elem(e).rjust(largura) for e, largura in zip(line, width))
        print("    │ " + formatted_line + " │")
    print("    └" + " " * nspaces + "┘")
    print()


In [4]:
r0 = 5

A = [cp.Parameter((2, 2), name='A[0]', value=np.array([[0, 1], [-1, 1]])),
     cp.Parameter((2, 2), name='A[1]', value=np.array([[0, 1], [-1, 1 - r0 ** 2]]))]
B = [cp.Parameter((2, 1), name='B[0]', value=np.array([[0], [1]])),
     cp.Parameter((2, 1), name='B[1]', value=np.array([[0], [1]]))]
I = cp.Parameter((2, 2), name='I', value=np.identity(2))

for i in range(len(A)):
  show_matrix(A[i].name(), A[i].value)

for i in range(len(B)):
  show_matrix(B[i].name(), B[i].value)

# Definição das Variáveis
Ξ_TIL = cp.Variable((2, 2), name='Ξ_TIL', PSD=True)
Ψ_TIL = cp.Variable((2, 2), name='Ψ_TIL', PSD=True)
X = cp.Variable((2, 2), name='X', PSD=True)
Q = cp.Variable((2, 2), name='Q', PSD=True)
K_TIL = [cp.Variable(
    (1, 2), name=f'K_TIL[{i}]') for i in range(2)]


def binary_to_decimal(binary_tuple):
  decimal_value = 0
  for bit in binary_tuple:
    decimal_value = (decimal_value << 1) | bit
  return decimal_value


# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(cp.trace(Ξ_TIL + Ψ_TIL + Q))

N11 = -Q
N12 = I
N21 = I.T
N22 = -X
N = cp.bmat([[N11, N12],
             [N21, N22]])

constraints = [N << 0]

BOOLEAN_DOMAIN = {0, 1}


def cartesian_power(set_, power):
  return set(product(set_, repeat=power))


p = 1
B_power = cartesian_power(BOOLEAN_DOMAIN, p)


def filter_cartesian_power(cartesian_set):
  filtered_set = set()
  for element in cartesian_set:
    valid = True
    for j in range(len(element) - 1):
      if element[j] > element[j+1]:
        valid = False
        break
    if valid:
      filtered_set.add(element)
  return filtered_set


B_p_plus = filter_cartesian_power(B_power)

permutations_B_p_plus = []

for m in B_p_plus:
  for n in B_p_plus:
    permutations_mn = set(permutations([m, n]))
    permutations_B_p_plus.append(permutations_mn)

display(permutations_B_p_plus)

conjuntos_simplificados = set(map(frozenset, permutations_B_p_plus))

for conjunto in conjuntos_simplificados:
  M = []
  for elemento in conjunto:
    bi, bj = elemento
    i, j = binary_to_decimal(bi), binary_to_decimal(bj)
    # if i == j: continue
    M11 = A[i] @ X + B[i] @ K_TIL[j] + X @ A[i].T + K_TIL[j].T @ B[i].T
    M12 = B[i] @ K_TIL[j]
    M13 = X

    M21 = K_TIL[j].T @ B[i].T
    M22 = -Ξ_TIL
    M23 = np.zeros(shape=(2, 2))

    M31 = X
    M32 = np.zeros(shape=(2, 2))
    M33 = -Ψ_TIL

    M += [cp.bmat([[M11, M12, M13],
                  [M21, M22, M23],
                  [M31, M32, M33]])]

  if M != []:
    constraints += [sum(M) << 0]

prob = cp.Problem(obj, constraints)

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)

# Apresentação dos Resultados
if prob.status not in ["infeasible", "unbounded"]:
  print("Valor ótimo: %s\n" % prob.value)
  for variable in prob.variables():
    if len(variable.shape) == 2:
      show_matrix(variable.name(), variable.value)
    else:
      print(variable.name(), '=', variable.value, '\n')

  X_INV = np.linalg.inv(X.value)
  Ξ = X_INV @ Ξ_TIL.value @ X_INV

  K0 = K_TIL[0] @ X_INV
  K1 = K_TIL[1] @ X_INV

  show_matrix('K[0]', K0.value)
  show_matrix('K[1]', K1.value)

  Ψ = np.linalg.inv(Ψ_TIL.value)

  show_matrix('P', X_INV)
  show_matrix('Ξ', Ξ),
  show_matrix('Ψ', Ψ),
else:
  print('O problema não é factível')

A[0] =
    ┌                     ┐
    │  0.00e+00  1.00e+00 │
    │ -1.00e+00  1.00e+00 │
    └                     ┘

A[1] =
    ┌                      ┐
    │  0.00e+00   1.00e+00 │
    │ -1.00e+00  -2.40e+01 │
    └                      ┘

B[0] =
    ┌          ┐
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘

B[1] =
    ┌          ┐
    │ 0.00e+00 │
    │ 1.00e+00 │
    └          ┘



[{((0,), (0,))},
 {((0,), (1,)), ((1,), (0,))},
 {((0,), (1,)), ((1,), (0,))},
 {((1,), (1,))}]

Valor ótimo: 7.8312831832349765

Ξ_TIL =
    ┌                      ┐
    │  2.72e-01  -8.70e-02 │
    │ -8.70e-02   1.93e+00 │
    └                      ┘

Ψ_TIL =
    ┌                      ┐
    │  1.12e+00  -2.06e-01 │
    │ -2.06e-01   5.90e-01 │
    └                      ┘

Q =
    ┌                    ┐
    │ 2.08e+00  4.10e-01 │
    │ 4.10e-01  1.83e+00 │
    └                    ┘

X =
    ┌                      ┐
    │  5.02e-01  -1.13e-01 │
    │ -1.13e-01   5.71e-01 │
    └                      ┘

K_TIL[1] =
    ┌                      ┐
    │ -2.70e+00  -1.04e+00 │
    └                      ┘

K_TIL[0] =
    ┌                     ┐
    │ 1.14e-01  -1.94e+00 │
    └                     ┘

K[0] =
    ┌                      ┐
    │ -5.59e-01  -3.50e+00 │
    └                      ┘

K[1] =
    ┌                      ┐
    │ -6.05e+00  -3.01e+00 │
    └                      ┘

P =
    ┌                    ┐
    │ 2.08e+00  4.10e-01 │
    │ 4.10e-01  1.83e+00 │
    └        

In [5]:
θo = (4. * math.pi) / 9.

A00 = cp.Parameter((2, 2), name='A[00]',
                   value=np.array([[0, 1], [math.sin(θo) / θo, -1]]))

A01 = cp.Parameter((2, 2), name='A[01]',
                   value=np.array([[0, 1], [math.sin(θo) / θo, -1]]))

A10 = cp.Parameter((2, 2), name='A[10]',
                   value=np.array([[0, 1], [1, -1]]))

A11 = cp.Parameter((2, 2), name='A[11]',
                   value=np.array([[0, 1], [1, -1]]))

B00 = cp.Parameter((2, 1), name='B[00]',
                   value=np.array([[0], [-math.cos(θo)]]))

B01 = cp.Parameter((2, 1), name='B[01]',
                   value=np.array([[0], [-1]]))


B10 = cp.Parameter((2, 1), name='B[10]',
                   value=np.array([[0], [-math.cos(θo)]]))

B11 = cp.Parameter((2, 1), name='B[11]',
                   value=np.array([[0], [-1]]))

A = [A00, A01, A10, A11]
B = [B00, B01, B10, B11]

I = cp.Parameter((2, 2), name='I', value=np.identity(2))

_ = [show_matrix(Ai.name(), Ai.value) for Ai in A]
_ = [show_matrix(Bi.name(), Bi.value) for Bi in B]

# Definição das Variáveis
Ξ_TIL = cp.Variable((2, 2), name='Ξ_TIL', PSD=True)
Ψ_TIL = cp.Variable((2, 2), name='Ψ_TIL', PSD=True)
X = cp.Variable((2, 2), name='X', PSD=True)
Q = cp.Variable((2, 2), name='Q', PSD=True)
K_TIL = [cp.Variable((1, 2), name='K_TIL[00]'),
         cp.Variable((1, 2), name='K_TIL[01]'),
         cp.Variable((1, 2), name='K_TIL[10]'),
         cp.Variable((1, 2), name='K_TIL[11]')]

# Definição do Problema: Objetivo e Restrições
obj = cp.Minimize(cp.trace(Ξ_TIL + Ψ_TIL + Q))

N11 = -Q
N12 = I
N21 = I.T
N22 = -X
N = cp.bmat([[N11, N12],
             [N21, N22]])

constraints = [N << 0]

r = 4
pairs = {(i, j) for i in range(1, r+1)
         for j in range(1, r+1) if i <= j}


for pair in pairs:
  i = pair[0] - 1
  j = pair[1] - 1

  if i == j:
    M11 = A[i] @ X + B[i] @ K_TIL[j] + X @ A[i].T + K_TIL[j].T @ B[i].T
    M12 = B[i] @ K_TIL[j]
    M13 = X

    M21 = K_TIL[j].T @ B[i].T
    M22 = -Ξ_TIL
    M23 = np.zeros(shape=(2, 2))

    M31 = X
    M32 = np.zeros(shape=(2, 2))
    M33 = -Ψ_TIL

    M = cp.bmat([[M11, M12, M13],
                 [M21, M22, M23],
                 [M31, M32, M33]])

    constraints += [M << 0]
  else:
    M11 = A[i] @ X + B[i] @ K_TIL[j] + X @ A[i].T + K_TIL[j].T @ B[i].T
    M12 = B[i] @ K_TIL[j]
    M13 = X

    M21 = K_TIL[j].T @ B[i].T
    M22 = -Ξ_TIL
    M23 = np.zeros(shape=(2, 2))

    M31 = X
    M32 = np.zeros(shape=(2, 2))
    M33 = -Ψ_TIL

    M = cp.bmat([[M11, M12, M13],
                 [M21, M22, M23],
                 [M31, M32, M33]])

    M11 = A[j] @ X + B[j] @ K_TIL[i] + X @ A[j].T + K_TIL[i].T @ B[j].T
    M12 = B[j] @ K_TIL[i]
    M13 = X

    M21 = K_TIL[i].T @ B[j].T
    M22 = -Ξ_TIL
    M23 = np.zeros(shape=(2, 2))

    M31 = X
    M32 = np.zeros(shape=(2, 2))
    M33 = -Ψ_TIL

    M += cp.bmat([[M11, M12, M13],
                  [M21, M22, M23],
                  [M31, M32, M33]])

    constraints += [M << 0]

prob = cp.Problem(obj, constraints)

print('n constraints:', len(constraints))

# Resolução do Problema usando o solver MOSEK
prob.solve(solver=cp.MOSEK, verbose=False)

# Apresentação dos Resultados
if prob.status not in ["infeasible", "unbounded"]:
  print("Valor ótimo: %s\n" % prob.value)
  # for variable in prob.variables():
  #   if len(variable.shape) == 2:
  #     show_matrix(variable.name(), variable.value)
  #   else:
  #     print(variable.name(), '=', variable.value, '\n')

  X_INV = np.linalg.inv(X.value)
  Ξ = X_INV @ Ξ_TIL.value @ X_INV

  K0 = K_TIL[0] @ X_INV
  K1 = K_TIL[1] @ X_INV
  K2 = K_TIL[2] @ X_INV
  K3 = K_TIL[3] @ X_INV

  show_matrix('K[00]', K0.value)
  show_matrix('K[01]', K1.value)
  show_matrix('K[10]', K2.value)
  show_matrix('K[11]', K3.value)

  Ψ = np.linalg.inv(Ψ_TIL.value)

  show_matrix('P', X_INV)
  show_matrix('Ξ', Ξ),
  show_matrix('Ψ', Ψ),
else:
  print('O problema não é factível')

A[00] =
    ┌                     ┐
    │ 0.00e+00   1.00e+00 │
    │ 7.05e-01  -1.00e+00 │
    └                     ┘

A[01] =
    ┌                     ┐
    │ 0.00e+00   1.00e+00 │
    │ 7.05e-01  -1.00e+00 │
    └                     ┘

A[10] =
    ┌                     ┐
    │ 0.00e+00   1.00e+00 │
    │ 1.00e+00  -1.00e+00 │
    └                     ┘

A[11] =
    ┌                     ┐
    │ 0.00e+00   1.00e+00 │
    │ 1.00e+00  -1.00e+00 │
    └                     ┘

B[00] =
    ┌           ┐
    │  0.00e+00 │
    │ -1.74e-01 │
    └           ┘

B[01] =
    ┌           ┐
    │  0.00e+00 │
    │ -1.00e+00 │
    └           ┘

B[10] =
    ┌           ┐
    │  0.00e+00 │
    │ -1.74e-01 │
    └           ┘

B[11] =
    ┌           ┐
    │  0.00e+00 │
    │ -1.00e+00 │
    └           ┘

n constraints: 11
Valor ótimo: 6.284573705307842

K[00] =
    ┌                    ┐
    │ 9.54e+00  6.28e+00 │
    └                    ┘

K[01] =
    ┌                    ┐
    │ 1.52e+00  1